In [ ]:
!pip install openai requests bs4 -q

In [ ]:
!pip install --upgrade openai

### Resources

- https://medium.com/@odhitom09/openai-responses-api-a-comprehensive-guide-ad546132b2ed

In [1]:
import requests
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
client = OpenAI(api_key="")

## Completions API

In [4]:
def scrapping_for(url: str):
    headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.title.string if soup.title else "No title found"
    
    for irrelevant in soup.body(["script", "style", "img", "input"]):
        irrelevant.decompose()
    
    text = soup.body.get_text(separator="\n", strip=True)

    return {
        "title": title,
        "text": text,
    }

def messages_for(website):
    system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

    user_prompt = f"You are looking at a website titled {website["title"]}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website["text"]
  
    
    return [
        {
            "role": "system",
            "content": system_prompt, 
        },
        {
            "role": "user",
            "content": user_prompt,             
        }
        
    ]

In [8]:
def summarize(url):    
    website = scrapping_for(url)
    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [9]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://medium.com/@johnidouglasmarangon")

# Summary of Johni Douglas Marangon – Medium

Johni Douglas Marangon is a tech enthusiast who shares insights and knowledge on various topics related to software development and artificial intelligence. The recent articles include:

1. **Scaling OpenAI API Usage with Multiple Python Requests** (6 days ago) - Discusses a service created to handle multiple prompts for a classification task.
   
2. **Efficient PDF Text Extraction in Java: OCR and Parsing Techniques** (Mar 7) - Explores methods for extracting text from PDF documents efficiently using Java.

3. **Generating Structured Outputs from PDFs with Semantic Kernel and Gemini** (Mar 3) - Addresses the need for structured data extraction from PDFs in modern applications.

4. **My Journey to Read the Vue.js 3 Official Documentation** (Feb 28) - Chronicles the author's experience with the Vue.js 3 documentation, focusing on the Getting Started guide and API.

5. **Integrating LLM into Java Applications with Spring AI** (Feb 14) - Covers the incorporation of Large Language Models into Java applications for enhancing NLP capabilities.

6. **Visualizing Data: Exploring Various Graphs in Python** (Feb 6) - Details the effectiveness of graphs and charts in interpreting data.

7. **OCR Tools: Solving Real-World Problems with Low-Quality Images** (Jan 29) - Investigates the application of Optical Character Recognition in real-world scenarios.

8. **Reversing Strings in Python: Immutability Explained** (Jan 23) - Explains the concept of string immutability in Python and its implications.

9. **Google Cloud Vision API for Image Handling and OCR** (Jan 17) - Introduces the Google Vision AI and its capabilities for image processing.

10. **Build a Speech-to-Text Service in Python with Faster Whisper** (Jan 11) - Shares insights from developing a speech-to-text system for court hearings in Brazil.

Johni aims to stay informed and share knowledge on the latest advancements in technology, especially in AI and web frameworks.

In [14]:
file1 = upload_file("slide.pdf")

In [15]:
response = make_question(file1, "Give me the generative AI tools")
display(Markdown(response))

The generative AI tools mentioned are:

1. **ChatGPT**
   - Developed by OpenAI (Nov. 2022)
   - [openai.com](https://chat.openai.com/)

2. **Bing AI**
   - Developed by Microsoft (Feb. 2023)
   - [bing.com](https://www.bing.com/)

3. **Google Bard**
   - Developed by Google (Mar. 2023)
   - [bard.google.com](https://bard.google.com/)

4. **Others**
   - Notion, Jasper, DALL-E, and many more.

## Examples using OpenAI Response API

#### Text Prompting - Instructions

In [21]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Talk like a math teacher.",
    input="Using only an addition, how do you add eight 8’s and get the number 1000?",
)

print(response.output_text)

Ah, this is a delightful little puzzle that involves some clever arrangement rather than straightforward arithmetic. To reach the number 1000 using eight 8’s, you can set them up like this:

888 + 88 + 8 + 8 + 8 = 1000

Notice how we’ve grouped and added the 8's together to cleverly achieve the desired sum. It's a great example of thinking outside the box!


#### Text Prompting - Chain of Command (hierarchical instructions)

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Talk like a teacher."},
        {"role": "developer", "content": "Explain the answer in detail using bullet points for each step."},
        {"role": "user", "content": "What is the result of the equation 5.93 * 10-2?"},
    ],
)

print(response.output_text)

#### Web Search

In [37]:

response = client.responses.create(
    model="gpt-4o-mini",
    input="What are the tarifs between the USA and China?",
    tools=[
        {
            "type" : "web_search_preview"
        }
    ]
)

print(response.output_text)

As of April 13, 2025, the trade relationship between the United States and China has been marked by escalating tariffs, significantly impacting various sectors.

**U.S. Tariffs on Chinese Imports:**

- **February 1, 2025:** The U.S. imposed a 10% tariff on all Chinese imports. ([koco.com](https://www.koco.com/article/china-retaliates-us-import-duties-trade-war/63759262?utm_source=openai))

- **March 4, 2025:** The tariff rate increased to 20%. ([dclcorp.com](https://dclcorp.com/blog/supply-chain/impact-us-china-tariffs/?utm_source=openai))

- **April 2, 2025:** An additional 34% "reciprocal tariff" was added, resulting in an effective minimum tariff of 54% on all imports from China. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration?utm_source=openai))

- **April 10, 2025:** The tariff rate further escalated to 145%. ([apnews.com](https://apnews.com/article/223e4cd53165da6fbd64aff5652a1aff?utm_source=openai))

**Chinese Retaliatory Tariffs:**



#### Reasoning

In [24]:
prompt = """
Write a recursive Python function to solve the Fibonacci problem.
Use memorization.
"""

response = client.responses.create(
    model="o3-mini",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}],
)

print(response.output_text)

Below is one way to write a recursive Fibonacci function using memoization (also called caching):

-------------------------------------------------------------
def fibonacci(n, memo=None):
    if memo is None:
        memo = {}
        
    # Base cases
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    # Check if result is already computed
    if n in memo:
        return memo[n]
    
    # Compute, store in memo, and return
    memo[n] = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
    return memo[n]

# Example usage:
print(fibonacci(10))  # Outputs: 55
-------------------------------------------------------------

Explanation:

1. The function fibonacci takes an integer n and an optional dictionary memo to store already computed values.
2. When memo is not provided, it gets initialized as an empty dictionary.
3. For the base cases (n == 0 and n == 1), the function returns 0 and 1, respectively.
4. Before performing recursive calls, the function checks if th

#### Structured Output - Whitout Parsing

In [31]:
import json
from typing import List

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": "Extract email and name from text.",
        },
        {
            "role": "user",
            "content": "Customer Alice Philip and her email alice@alice.com.",
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "person_info",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "email": {"type": "string"},
                },
                "required": ["name", "email"],
                "additionalProperties": False,
            },
            "strict": True,
        }
    },
)

event = json.loads(response.output_text)

print(event)

{'name': 'Alice Philip', 'email': 'alice@alice.com'}


### Structured Output - With Parsing

In [34]:
from pydantic import BaseModel

class PersonInfo(BaseModel):
    name: str
    email: str


response = client.responses.parse(
    model="gpt-4o",
    instructions="Extract email and name from text.",
    input="Customer Alice Philip and her email alice@alice.com.",
    text_format=PersonInfo,
)

response_model = response.output[0].content[0].parsed

print(type(response_model))
print(response_model.model_dump_json(indent=2))

<class '__main__.PersonInfo'>
{
  "name": "Alice Philip",
  "email": "alice@alice.com"
}


### Function Calling 

In [35]:
tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "The recipient email address."},
                "subject": {"type": "string", "description": "Email subject line."},
                "body": {"type": "string", "description": "Body of the email message."},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

response = client.responses.create(
    model="gpt-4o",
    input="Can you send an email to ilan@example.com and katia@example.com saying hi?",
    tools=tools,
)

print(response.output)
print(response.output[0].model_dump_json(indent=2))
print(response.output[1].model_dump_json(indent=2))

[ResponseFunctionToolCall(arguments='{"to":"ilan@example.com","subject":"Hello","body":"Hi Ilan!"}', call_id='call_Gmqd9UGZFa36mKey1Jtfb4To', name='send_email', type='function_call', id='fc_67fbc9ae72b08192b9b1b50b0e4631270ffcfd41484612d1', status='completed'), ResponseFunctionToolCall(arguments='{"to":"katia@example.com","subject":"Hello","body":"Hi Katia!"}', call_id='call_K6kqwTaoe6OTubulv3s9tQ9U', name='send_email', type='function_call', id='fc_67fbc9aeaca4819286956a00a1655dd20ffcfd41484612d1', status='completed')]
{
  "arguments": "{\"to\":\"ilan@example.com\",\"subject\":\"Hello\",\"body\":\"Hi Ilan!\"}",
  "call_id": "call_Gmqd9UGZFa36mKey1Jtfb4To",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_67fbc9ae72b08192b9b1b50b0e4631270ffcfd41484612d1",
  "status": "completed"
}
{
  "arguments": "{\"to\":\"katia@example.com\",\"subject\":\"Hello\",\"body\":\"Hi Katia!\"}",
  "call_id": "call_K6kqwTaoe6OTubulv3s9tQ9U",
  "name": "send_email",
  "type": "function_call",
  

#### File Search

In [13]:
uploaded_file = client.files.create(
        file=open(filename, "rb"),
        purpose="user_data",
)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": uploaded_file.id,
                },
                {
                    "type": "input_text",
                    "text": question,
                },
            ]
        }
    ]
)

pritn(response.output_text)